# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 3 2022 9:08AM,249948,12,HH-37931,Hush Hush,Released
1,Nov 3 2022 9:08AM,249948,12,HH-37932,Hush Hush,Released
2,Nov 3 2022 9:08AM,249948,12,HH-37933,Hush Hush,Released
3,Nov 3 2022 9:08AM,249948,12,HH-37934,Hush Hush,Released
4,Nov 3 2022 9:08AM,249948,12,HH-37935,Hush Hush,Released
5,Nov 3 2022 9:08AM,249948,12,HH-37936,Hush Hush,Released
6,Nov 3 2022 9:08AM,249948,12,HH-37938,Hush Hush,Released
7,Nov 3 2022 9:08AM,249948,12,HH-37939,Hush Hush,Released
8,Nov 3 2022 9:08AM,249948,12,HH-37940,Hush Hush,Released
9,Nov 3 2022 8:54AM,249946,10,0086139716,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,249941,Released,1
30,249944,Released,41
31,249945,Released,1
32,249946,Released,11
33,249948,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249941,NaN,3.0,1.0
249944,NaN,NaN,41.0
249945,NaN,NaN,1.0
249946,NaN,NaN,11.0
249948,NaN,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249856,26.0,1.0,1.0
249880,0.0,0.0,1.0
249881,0.0,0.0,1.0
249884,0.0,0.0,1.0
249885,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249856,26,1,1
249880,0,0,1
249881,0,0,1
249884,0,0,1
249885,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249856,26,1,1
1,249880,0,0,1
2,249881,0,0,1
3,249884,0,0,1
4,249885,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249856,26,1,1
1,249880,,,1
2,249881,,,1
3,249884,,,1
4,249885,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 3 2022 9:08AM,249948,12,Hush Hush
9,Nov 3 2022 8:54AM,249946,10,ISDIN Corporation
20,Nov 3 2022 8:48AM,249945,12,"NBTY Global, Inc."
21,Nov 3 2022 8:25AM,249944,10,ISDIN Corporation
62,Nov 3 2022 8:06AM,249941,10,ISDIN Corporation
66,Nov 3 2022 7:43AM,249938,19,"AdvaGen Pharma, Ltd"
70,Nov 2 2022 4:41PM,249933,15,"Person & Covey, Inc."
98,Nov 2 2022 4:11PM,249930,19,"AdvaGen Pharma, Ltd"
112,Nov 2 2022 3:57PM,249928,19,"Innogenix, LLC"
117,Nov 2 2022 3:48PM,249927,19,"Innogenix, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 3 2022 9:08AM,249948,12,Hush Hush,,,9
1,Nov 3 2022 8:54AM,249946,10,ISDIN Corporation,,,11
2,Nov 3 2022 8:48AM,249945,12,"NBTY Global, Inc.",,,1
3,Nov 3 2022 8:25AM,249944,10,ISDIN Corporation,,,41
4,Nov 3 2022 8:06AM,249941,10,ISDIN Corporation,,3,1
5,Nov 3 2022 7:43AM,249938,19,"AdvaGen Pharma, Ltd",,,4
6,Nov 2 2022 4:41PM,249933,15,"Person & Covey, Inc.",,,28
7,Nov 2 2022 4:11PM,249930,19,"AdvaGen Pharma, Ltd",,,14
8,Nov 2 2022 3:57PM,249928,19,"Innogenix, LLC",,4,1
9,Nov 2 2022 3:48PM,249927,19,"Innogenix, LLC",,4,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 9:08AM,249948,12,Hush Hush,9,,
1,Nov 3 2022 8:54AM,249946,10,ISDIN Corporation,11,,
2,Nov 3 2022 8:48AM,249945,12,"NBTY Global, Inc.",1,,
3,Nov 3 2022 8:25AM,249944,10,ISDIN Corporation,41,,
4,Nov 3 2022 8:06AM,249941,10,ISDIN Corporation,1,3,
5,Nov 3 2022 7:43AM,249938,19,"AdvaGen Pharma, Ltd",4,,
6,Nov 2 2022 4:41PM,249933,15,"Person & Covey, Inc.",28,,
7,Nov 2 2022 4:11PM,249930,19,"AdvaGen Pharma, Ltd",14,,
8,Nov 2 2022 3:57PM,249928,19,"Innogenix, LLC",1,4,
9,Nov 2 2022 3:48PM,249927,19,"Innogenix, LLC",1,4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 9:08AM,249948,12,Hush Hush,9,,
1,Nov 3 2022 8:54AM,249946,10,ISDIN Corporation,11,,
2,Nov 3 2022 8:48AM,249945,12,"NBTY Global, Inc.",1,,
3,Nov 3 2022 8:25AM,249944,10,ISDIN Corporation,41,,
4,Nov 3 2022 8:06AM,249941,10,ISDIN Corporation,1,3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 9:08AM,249948,12,Hush Hush,9,NaN,NaN
1,Nov 3 2022 8:54AM,249946,10,ISDIN Corporation,11,NaN,NaN
2,Nov 3 2022 8:48AM,249945,12,"NBTY Global, Inc.",1,NaN,NaN
3,Nov 3 2022 8:25AM,249944,10,ISDIN Corporation,41,NaN,NaN
4,Nov 3 2022 8:06AM,249941,10,ISDIN Corporation,1,3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 3 2022 9:08AM,249948,12,Hush Hush,9,0.0,0.0
1,Nov 3 2022 8:54AM,249946,10,ISDIN Corporation,11,0.0,0.0
2,Nov 3 2022 8:48AM,249945,12,"NBTY Global, Inc.",1,0.0,0.0
3,Nov 3 2022 8:25AM,249944,10,ISDIN Corporation,41,0.0,0.0
4,Nov 3 2022 8:06AM,249941,10,ISDIN Corporation,1,3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,749831,53,3.0,0.0
12,499893,10,0.0,0.0
15,499789,29,1.0,26.0
16,499772,2,0.0,0.0
19,1499573,33,22.0,0.0
21,2998709,12,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,749831,53,3.0,0.0
1,12,499893,10,0.0,0.0
2,15,499789,29,1.0,26.0
3,16,499772,2,0.0,0.0
4,19,1499573,33,22.0,0.0
5,21,2998709,12,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,53,3.0,0.0
1,12,10,0.0,0.0
2,15,29,1.0,26.0
3,16,2,0.0,0.0
4,19,33,22.0,0.0
5,21,12,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,53.0
1,12,Released,10.0
2,15,Released,29.0
3,16,Released,2.0
4,19,Released,33.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,0.0,0.0,26.0,0.0,0.0,0.0
Executing,3.0,0.0,1.0,0.0,22.0,0.0
Released,53.0,10.0,29.0,2.0,33.0,12.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,26.0,0.0,0.0,0.0
1,Executing,3.0,0.0,1.0,0.0,22.0,0.0
2,Released,53.0,10.0,29.0,2.0,33.0,12.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,26.0,0.0,0.0,0.0
1,Executing,3.0,0.0,1.0,0.0,22.0,0.0
2,Released,53.0,10.0,29.0,2.0,33.0,12.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()